### Definição dos objetos
Inicialmente será criado as classes fundamentais dos objetos que iremos trabalhar

A classe Categoria representa as categorias existentes de equipamentos de TI
A classe SistemOp represeta os tipos de sistema operacional
A classe Equipamento representa os equipamentos de classe Categoria e SistemaOp

In [57]:
from enum import Enum


class Categoria(Enum):
    NOTEBOOK = 1
    DESKTOP = 2
    SERVIDOR = 3
    VM = 4
    
class SistemaOp(Enum):
    WINDOWS = 1
    LINUX = 2
    

class Equipamento:
    def __init__(self, name, sistema, categoria):
        self.name = name
        self.sistema = sistema
        self.categoria = categoria
        


### Criação dos filtros
Será criado duas classes genéricas Especificacao e Filtro.

A classe Especificacao é responsável por tornar genérica as especificações, possibilitando todas as especificações novas serem herdeiras dela.

A classe Filtro é responsável por tornar genérico os filtros, possibilitando todos os novos filtros serem herdeiros de Filtro

In [58]:

class Especificacao:
    def is_satisfied(self, equipamento):
        pass

    # and operator makes life easier
    def __and__(self, other):
        return AndEspecificacao(self, other)


class Filtro:
    def filtro(self, equipamentos, spec):
        pass

    
class CategoriaEspecificacao(Especificacao):
    def __init__(self, categoria):
        self.categoria = categoria
        
    def is_satisfied(self, equipamento):
        return equipamento.categoria == self.categoria
    
    
class SistemaOpEspecificacao(Especificacao):
    def __init__(self, sistema):
        self.sistema = sistema
        
    def is_satisfied(self, equipamento):
        return equipamento.sistema == self.sistema
    

class FiltroIter(Filtro):
    '''
        Realiza o filtro com uma determinada especificacao iterando sobre os objetos
    '''
    def filtro(self, equipamentos, spec):
        for eqp in equipamentos:
            print(eqp.name)
            if spec.is_satisfied(eqp):
                yield eqp

### Criação dos Equipamentos

In [60]:
notebook = Equipamento('Acer Aspire VX', SistemaOp.WINDOWS, Categoria.NOTEBOOK)
servidor = Equipamento('SGGXP00123VIX', SistemaOp.LINUX, Categoria.SERVIDOR)
vm1 = Equipamento('SGGXP00121CLD', SistemaOp.LINUX, Categoria.VM)
vm2 = Equipamento('SGGXP00122CLD', SistemaOp.LINUX, Categoria.VM)

equipamentos = [notebook, servidor, vm1, vm2]

### Filtro dos equipamentos

In [61]:
f = FiltroIter()

In [62]:
print('Equipamentos Windows:')
windows = SistemaOpEspecificacao(SistemaOp.WINDOWS)
for p in f.filtro(equipamentos, windows):
    print(f' - {p.name} is windows')

Equipamentos Windows:
Acer Aspire VX
 - Acer Aspire VX is windows
SGGXP00123VIX
SGGXP00121CLD
SGGXP00122CLD


In [63]:
print('Equipamentos Notebook:')
notebook = CategoriaEspecificacao(Categoria.NOTEBOOK)
for p in f.filtro(equipamentos, notebook):
    print(f' - {p.name} is notebook')

Equipamentos Notebook:
Acer Aspire VX
 - Acer Aspire VX is notebook
SGGXP00123VIX
SGGXP00121CLD
SGGXP00122CLD


### E se realizarmos quisermos filtrar pro mais de uma categoria?

#### Conceitos Básicos (map, all, lambda)

**map**

*map aplica uma determinada função em cada elemento da lista*

In [100]:
import math
nums = [1,4,9]
list(map(math.sqrt, nums))

[1.0, 2.0, 3.0]

**all**

*Retorna True se todos os elementos do iterável forem verdadeiros*

In [101]:
listaDeInteiros = [2, 6, 4, 7, -2]
if all(i % 2 == 0 for i in listaDeInteiros):
    print('Todos sao pares')
else:
    print('Tem algum impar')

Tem algum impar


**lambda**

*Definição de funções anonimas, ou seja, que não precisam ser explicitamente declaradas*

In [102]:
(lambda x,y: x+y)(1,2)

3

#### Criação do filtro AND
O objetivo desse filtro, é realizar o AND entre todas as especificações, retornando apenas os dispositivos com todas as características desejadas

In [5]:
class AndEspecificacao(Especificacao):
    def __init__(self, *args):
        self.args = args

    def is_satisfied(self, item):
        return all(map(
            lambda espec: espec.is_satisfied(item), self.args))

In [6]:
windows = SistemaOpEspecificacao(SistemaOp.WINDOWS)
notebook = CategoriaEspecificacao(Categoria.NOTEBOOK)
windows_notebook = AndEspecificacao(windows, notebook)

for p in f.filtro(equipamentos, windows_notebook):
    print(f' - {p.name} is notebook')

Acer Aspire VX
 - Acer Aspire VX is notebook
SGGXP00123VIX
SGGXP00121CLD
SGGXP00122CLD


### Legal, agora seu gerente de projetos solicitou mais um tipo de filtro. Além das classificações existentes, teremos um novo tipo de Servidor na AWS, com a caracteristica de "Tamanho". 

### Consequentemente, o desejo agora é filtrar por tamanho de máquina, quando forem equipamentos da AWS.  Qual a melhor forma de realizar o procedimento?

Visto que as funcionalidades acimas já foram validadas e testadas, elas não devem mais ser alteradas. É por isso que a generalização é importante.

Veja como podemos realizar a adiação desse filtro, sem alterar as features já validadas

In [52]:
class EquipamentoAws(Equipamento):
    def __init__(self, name, sistema, categoria, tamanho):
        super().__init__(name, sistema, categoria)
        self.tamanho = tamanho #Campo Adicionado
        
class Tamanho(Enum):
    GRANDE = 1
    MEDIO = 2
    PEQUENO = 3
    
    
class TamanhoEspecificacao(Especificacao):
    def __init__(self, tamanho):
        self.tamanho = tamanho
        
    def is_satisfied(self, equipamento):
        return equipamento.tamanho == self.tamanho

In [68]:
servidor_aws = EquipamentoAws('SGGXP00345AWS', SistemaOp.WINDOWS, Categoria.SERVIDOR,
                              Tamanho.GRANDE)
vm_aws = EquipamentoAws('SGGXP00451AWS', SistemaOp.LINUX, Categoria.VM,
                               Tamanho.MEDIO)


equipamentos = [notebook, servidor, vm1, vm2, servidor_aws, vm_aws]
equipamentos_aws = [servidor_aws, vm_aws]

In [113]:
class FiltroAws(FiltroIter):
        
        
    def filtro(self, equipamentos, espec):
#         equipamentos_aws = [eqp for eqp in equipamentos 
#                                 if type(eqp).__name__ == 'EquipamentoAws']

        super().filtro(equipamentos, espec_filtro)
#         for eqp in equipamentos:
#             print(eqp.name)
#             if espec.is_satisfied(eqp):
#                 yield eqp
    

In [114]:
windows = SistemaOpEspecificacao(SistemaOp.WINDOWS)
servidor = CategoriaEspecificacao(Categoria.SERVIDOR)
grande = TamanhoEspecificacao(Tamanho.GRANDE)
espec_filtro = AndEspecificacao(windows, servidor, grande)

f_aws = FiltroAws()

print(f_aws.filtro(equipamentos_aws, espec_filtro))

for p in f_aws.filtro(equipamentos_aws, espec_filtro):
    print(f' - {p.name} is AWS, Large')

None


TypeError: 'NoneType' object is not iterable

In [37]:
espec_filtro

In [34]:
[eqp for eqp in equipamentos if type(eqp).__name__ == 'EquipamentoAws']